In [52]:
import climlab
from climlab.process import TimeDependentProcess
import pandas as pd
import xarray as xr
import numpy as np
from attrdict import AttrDict


In [54]:
#create a turbulence class
class Turbulence(TimeDependentProcess):
    #class turbulence that decays with height computed from:
    # theta = Tatm(P0/Patm)^R/Cp
    # k0 = Net surface flux/(dtheta/dz) at surface 
    # ka = k0*exp(-z/d) in atmosphere
    # -ka * dtheta/dz is turbulent flux in the atmosphere 
    
    def __init__(self, **kwargs):
        #define initial state
        super(Turbulence, self).__init__(**kwargs)
        self.time_type = 'explicit'
        self._hr = {}
        for var in self.state:
            self._hr[var] = 0. *self.state[var]
        ### define new levels
        #n+1 level (lev_bounds) reworked so that we are no longer using 0 and 1000 as our TOA and surface
        self.lev_bounds[0] = self.lev_bounds[1]-2*(self.lev_bounds[1] - self.lev[0])
        self.lev_bounds[-1] = self.lev_bounds[-2]+2*(self.lev[-1] - self.lev_bounds[-2])
        #n+2 level (lev_pad)
        self.lev_pad = np.zeros(len(state['Tatm']) + 2)
        self.lev_pad[1:] = self.lev_bounds
        self.lev_pad[0] = self.lev_bounds[0]-2*(self.lev_bounds[1] - self.lev_bounds[0])
        ### calculate turbulent flux
        #surface flux (LW + SW)
        self._total_sfc_flux_init = (rad.diagnostics['LW_flux_net_clr'] + rad.diagnostics['SW_flux_net_clr'])[-1]
        #gas constant / specific heat of air
        R_cp = 0.286 
        #theta (length of n)
        self._theta_init = self.state['Tatm']*((self.lev[0]/self.lev)**(R_cp)) 
        #dtheta_dz_init (length of n+1 by hard coding in our top and bottom levels)
        self._dtheta_dz_init = np.zeros(len(state['Tatm']) +1)
        self._dtheta_dz_init[1:-1] = np.diff(self._theta_init)/np.diff(self.lev)
        self._dtheta_dz_init[0] =  0 #### USE THE TOP DTHETA_DZ TWICE
        self._dtheta_dz_init[-1] =  -self._total_sfc_flux_init
        #surface diffk
        self._surface_diffk = self._total_sfc_flux_init/self._dtheta_dz_init[-2]
        #atmospheric diffk
        scale_factor = 100
        k1 = -8.78700 #m/mb
        k0 = 8893 #m
        self._alt_linear_est = k1 * self.lev_bounds + k0 
        self._atm_diffk = self._surface_diffk * (np.exp(-self._alt_linear_est/scale_factor)) #### CHECK IF GOOD FOR ALT #length scale of 100
        

    def _compute(self):
        #surface flux (LW + SW)
        self._total_sfc_flux = (rad.diagnostics['LW_flux_net_clr'] + rad.diagnostics['SW_flux_net_clr'])[-1]
        #gas constant / specific heat of air
        R_cp = 0.286 
        #theta (length of n)
        self._theta = self.state['Tatm']*((self.lev[0]/self.lev)**(R_cp)) 
        #dtheta_dz_init (length of n+1)
        self._dtheta_dz = np.zeros(len(state['Tatm']) +1)
        self._dtheta_dz[1:-1] = np.diff(self._theta)/np.diff(self.lev)
        self._dtheta_dz[0] =  0 #### USE THE TOP DTHETA_DZ TWICE
        self._dtheta_dz[-1] =  -self._total_sfc_flux
        #calculate the turbulent flux
        self._turbulent_flux = -self._atm_diffk * self._dtheta_dz
        # calculate heating rate (flux convergence) from flux
        self._hr = (np.diff(self._turbulent_flux)/np.diff(self.lev_bounds))
        tendencies = {'Tatm' : self._hr}
        return tendencies

In [1]:
#test the turbulence class

alb = 0.8
#  State variables (Air and surface temperature)
state =climlab.column_state(num_lev=96)
#  Fixed relative humidity
h2o = climlab.radiation.ManabeWaterVapor(name='WaterVapor', state=state)
#  Couple water vapor to radiation
rad = climlab.radiation.RRTMG(name='Radiation', state=state, specific_humidity=h2o.q, albedo=alb)
#  Couple turbulence to radiation
turb = Turbulence(name = 'Turbulence', state=state, rad = rad)
#  Couple everything together
rcm = climlab.TimeDependentProcess(state = state)
rcm.add_subprocess('Turbulence', turb) #add insolation subprocess
rcm.add_subprocess('Radiation', rad) #add insolation subprocess
rcm.add_subprocess('WaterVapor', h2o) #add insolation subprocess
rcm.compute()

NameError: name 'climlab' is not defined